In [1]:
# https://layoffs.fyi/
# https://www.kaggle.com/code/debjeetdas/exploratory-data-analysis-on-layoffs-dataset-2023?kernelSessionId=117955624

In [36]:
import os

import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [37]:
all_graphs = []

In [38]:
df = pd.read_csv('layoffs_data.csv', parse_dates=['Date'])
#drop the below cols as they are not useful for the analysis
df.drop(['Source','Date_Added','List_of_Employees_Laid_Off'],axis=1,inplace=True)
df.sort_values('Date',inplace=True)
df.head()

,Company,Location_HQ,Industry,Percentage,Date,Funds_Raised,Stage,Country,Laid_Off_Count
2191,EasyPost,Salt Lake City,Logistics,NaN,2020-03-11,12.0,Series A,United States,75.0
2190,Tamara Mellon,Los Angeles,Retail,0.40,2020-03-12,90.0,Series C,United States,20.0
2188,HopSkipDrive,Los Angeles,Transportation,0.10,2020-03-13,45.0,Unknown,United States,8.0
2189,Panda Squad,SF Bay Area,Consumer,0.75,2020-03-13,1.0,Seed,United States,6.0
2186,Help.com,Austin,Support,1.00,2020-03-16,6.0,Seed,United States,16.0


In [39]:
#total company participated in layoff
df['Company'].nunique()

1777

In [40]:
#total employees laid off
df['Laid_Off_Count'].sum()

352115.0

In [41]:
data_from_to_date = str(df['Date'].min().date())+' to '+str(df['Date'].max().date())
data_from_to_date

'2020-03-11 to 2023-02-07'

In [42]:
#total company participated in layoff
total_companies_layoff = df['Company'].nunique()

#total employees laid off
total_employees_layoff = df['Laid_Off_Count'].sum()

In [43]:
df['Year'] = df['Date'].dt.year
# df['Year'] = df['Year'].astype(str)

In [44]:
yearwise_laidoff = df.groupby('Year')['Laid_Off_Count'].sum().reset_index()
yearwise_laidoff['Year'] = yearwise_laidoff['Year'].astype(str)
yearwise_laidoff

,Year,Laid_Off_Count
0,2020,80998.0
1,2021,15823.0
2,2022,159786.0
3,2023,95508.0


In [45]:
#year wise trend
yearwise_trend = px.area(yearwise_laidoff, x='Year', y='Laid_Off_Count',markers=True, \
              title='Yearwise No. of layoffs',template="plotly_dark")
yearwise_trend.show()

yearly_layoff_perc = px.pie(yearwise_laidoff, names='Year', values='Laid_Off_Count', title='Yearly Layoff percentage',\
                            color_discrete_sequence=px.colors.sequential.RdBu,template="plotly_dark")
yearly_layoff_perc.show()

all_graphs.extend([yearwise_trend, yearly_layoff_perc])

In [46]:
#date interval wise employeess laidoff
date_intervalwise_laidoff = px.histogram(df.dropna(subset=['Laid_Off_Count']), x='Date', y='Laid_Off_Count', histfunc='sum', text_auto=True,template="plotly_dark")
date_intervalwise_laidoff.update_xaxes(categoryorder='total descending')
date_intervalwise_laidoff.show()

# all_graphs.append(date_intervalwise_laidoff)

In [47]:
#industry wise laid off
df['Industry'].replace({'Unknown':'Other'}, inplace=True)
industrywise_laidoff = px.histogram(df.dropna(subset=['Laid_Off_Count']), x='Industry', y='Laid_Off_Count', histfunc='sum', \
                   text_auto=True, title='industry wise layoffs',color_discrete_sequence=px.colors.sequential.RdBu,template="plotly_dark")
industrywise_laidoff.update_xaxes(categoryorder='total descending')
industrywise_laidoff.show()

all_graphs.append(industrywise_laidoff)

In [48]:
### continent wise layoffs
continent = pd.read_csv('continents2.csv')[['name','alpha-3','region']]
continent = continent.set_index('name')

continent = continent.to_dict()#['region']
df['Country_iso_code'] = df['Country'].map(continent['alpha-3'])
df['Continent'] = df['Country'].map(continent['region'])

### continent wise layoff
continentwise_layoff = px.histogram(df.dropna(subset=['Laid_Off_Count']), y='Continent', x='Laid_Off_Count', histfunc='sum', \
                   text_auto=True, title='Continent wise layoffs',color_discrete_sequence=px.colors.sequential.RdBu,template="plotly_dark")
continentwise_layoff.update_yaxes(categoryorder='total ascending')
continentwise_layoff.show()

all_graphs.append(continentwise_layoff)

In [49]:
# year wise#country wise layoff
# country_wise_layoff = df.groupby(['Country','Country_iso_code','Year'])['Laid_Off_Count'].sum().sort_index(level=[0,1,2])\
# .groupby(level=0).cumsum().reset_index()
# country_wise_layoff.sort_values(by='Year',inplace=True)
# #country wise layoffs
# countrywise_layoff = px.choropleth(country_wise_layoff, locations='Country_iso_code', hover_name='Country', color='Laid_Off_Count',\
#               animation_frame = 'Year',\
#               range_color=(country_wise_layoff['Laid_Off_Count'].min(),country_wise_layoff['Laid_Off_Count'].max()),\
#               color_continuous_scale=px.colors.sequential.Plasma[::-1], title='Yearly Country wise Cumulative LayOffs')
# # fig.write_image('yes.png')
# countrywise_layoff.show(renderer="colab")

# all_graphs.append(countrywise_layoff)

#overall country wise layoff
country_wise_layoff = df.groupby(['Country','Country_iso_code'])['Laid_Off_Count'].sum().sort_index(level=[0,1]).reset_index()
# country_wise_layoff.sort_values(by='Year',inplace=True)
#country wise layoffs
countrywise_layoff = px.choropleth(country_wise_layoff, locations='Country_iso_code', hover_name='Country', color='Laid_Off_Count',\
              range_color=(country_wise_layoff['Laid_Off_Count'].min(),country_wise_layoff['Laid_Off_Count'].max()),\
              color_continuous_scale=px.colors.sequential.Turbo, title='Country wise Total LayOffs',template="plotly_dark")
# fig.write_image('yes.png')
countrywise_layoff.show(renderer="colab")

all_graphs.append(countrywise_layoff)

In [50]:
country_wise_layoff = country_wise_layoff.merge(df[['Country_iso_code','Continent']], on='Country_iso_code')
country_wise_layoff

,Country,Country_iso_code,Laid_Off_Count,Continent
0,Argentina,ARG,323.0,Americas
1,Argentina,ARG,323.0,Americas
2,Argentina,ARG,323.0,Americas
3,Argentina,ARG,323.0,Americas
4,Argentina,ARG,323.0,Americas
...,...,...,...,...
2187,United States,USA,240042.0,Americas
2188,Uruguay,URY,0.0,Americas
2189,Vietnam,VNM,0.0,Asia
2190,Vietnam,VNM,0.0,Asia


In [51]:
continents = country_wise_layoff.Continent.unique()#.values
# print(continents.shape)
continent_country_layoffs = make_subplots(rows=continents.shape[0]//2+continents.shape[0]%2, cols=2,\
                    subplot_titles=continents)
# print(fig)
i,j=1,1
for cont in continents:
    tmp_df = country_wise_layoff[country_wise_layoff['Continent']==cont]
    tmp_df = tmp_df.groupby('Country')['Laid_Off_Count'].sum().reset_index()
    continent_country_layoffs.add_trace(
        go.Bar(x=tmp_df['Country'].values, y=tmp_df['Laid_Off_Count'].values,text=tmp_df['Laid_Off_Count']),
        row=i, col=j
    )
    j+=1
    if j==3:
      i+=1
      j=1


continent_country_layoffs.update_layout(height=800, width=1000, title_text="Continets & Country wise Layoffs",showlegend=False)
continent_country_layoffs.show()

all_graphs.append(continent_country_layoffs)

## 100% layoffs company

In [52]:
lay_100 = df[df['Percentage']==1]

### continent layoff 100% employees
continent100perc_layoff = px.histogram(lay_100.dropna(subset=['Laid_Off_Count']), y='Company', x='Laid_Off_Count', histfunc='sum', \
                   text_auto=True, title='Company layoffs 100% employees',template="plotly_dark")
continent100perc_layoff.update_yaxes(categoryorder='total ascending')
continent100perc_layoff.show()

# all_graphs.append(continent100perc_layoff)

In [53]:
country_100per_layoff = lay_100.groupby('Country')['Laid_Off_Count'].sum().reset_index()
country_100per_layoff = country_100per_layoff[country_100per_layoff['Laid_Off_Count']>0]

country100perc_layoff_fig = px.pie(country_100per_layoff, names='Country', values='Laid_Off_Count',title='Top Countries in 100% layoffs',template="plotly_dark")
country100perc_layoff_fig.show()

# all_graphs.append(country100perc_layoff_fig)

#2023 top companies who layoff the most

In [54]:

top_companies2023 = df[df['Year']==2023]
top_companies2023 = top_companies2023.groupby('Company')['Laid_Off_Count'].sum().reset_index()\
                    .sort_values(by='Laid_Off_Count',ascending=False).reset_index(drop=True).iloc[:10,:]

### Top companies in layoffs
top_companies2023_fig = px.histogram(top_companies2023, y='Company', x='Laid_Off_Count', histfunc='sum', \
                   text_auto=True, title='Top Companies layoffs most employees in 2023',color_discrete_sequence=px.colors.sequential.RdBu,template="plotly_dark")
top_companies2023_fig.update_yaxes(categoryorder='total ascending')
top_companies2023_fig.show()


all_graphs.append(top_companies2023_fig)

In [55]:
top_companies_details = df.loc[(df['Company'].isin(top_companies2023['Company'].values)) & (df['Year']==2023)]
top_companies_details.sort_values(by='Laid_Off_Count',ascending=False,inplace=True)

for company, percentage, total in zip(top_companies_details['Company'],top_companies_details['Percentage'],\
                               top_companies_details['Laid_Off_Count']):
  print(f'{company} laidoff {round(percentage*100,2)}% employees. => {total}')

Google laidoff 6.0% employees. => 12000.0
Microsoft laidoff 5.0% employees. => 10000.0
Amazon laidoff 2.0% employees. => 8000.0
Salesforce laidoff 10.0% employees. => 8000.0
Dell laidoff 5.0% employees. => 6650.0
Philips laidoff 13.0% employees. => 6000.0
IBM laidoff 2.0% employees. => 3900.0
SAP laidoff 3.0% employees. => 3000.0
PayPal laidoff 7.0% employees. => 2000.0
Wayfair laidoff 10.0% employees. => 1750.0


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Deploy on plotly Dash

In [56]:
!pip install Dash
!pip install dash_bootstrap_components
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
len(all_graphs)

7

In [58]:
# # Run this app with `python app.py` and
# # visit http://127.0.0.1:8050/ in your web browser.
# from jupyter_dash import JupyterDash
# from dash import Dash, html, dcc
# import plotly.express as px
# import pandas as pd
# import dash_bootstrap_components as dbc

# app = JupyterDash(__name__,external_stylesheets=[
#     "https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap-grid.min.css"
# ])

# # assume you have a "long-form" data frame
# # see https://plotly.com/python/px-arguments/ for more options
# df = pd.DataFrame({
#     "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
#     "Amount": [4, 1, 2, 2, 4, 5],
#     "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
# })

# # card = dbc.Card([
# #     dbc.CardBody([
# #         html.H4('yes did it',className='card-title')
# #     ])
# # ],style={"width": "18rem"})

# card = dbc.Card(html.P('yes did it'),body=True)

# fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

# rows_list = []
# for i in range(0,len(all_graphs),2):
#   rows_list.append(dbc.Row([
#       dbc.Col(dcc.Graph(figure=all_graphs[i])),
#       dbc.Col(dcc.Graph(figure=all_graphs[i+1])) if i+1 < len(all_graphs) else None
#       ])
#   )

# app.layout = html.Div(children=[
#     html.H1(children='Hello Dash'),

#     html.Div(children='''
#         Dash: A web application framework for your data.
#     '''),
#     card,
#     html.Div(
#       rows_list
#     )
#   ])

# #justify -> start, center, end, between, around
# if __name__ == '__main__':
#     app.run_server(mode='external',host="0.0.0.0",port=1005,debug=False)


# plotly dash deployment

In [81]:
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

# from jupyter_dash import JupyterDash
# from dash import Dash, html, dcc
# import plotly.express as px
# import pandas as pd
# import dash_bootstrap_components as dbc

app = JupyterDash(__name__,external_stylesheets=[
    dbc.themes.SPACELAB, dbc.icons.BOOTSTRAP, dbc.themes.DARKLY
])

card = lambda title, value: dbc.Card(
    dbc.CardBody([
          html.H2(title),
          html.H3(value, style={'color':'red'})
    ],
    className='text-center'
    )
)

# rows_list = []
# for i in range(0,len(all_graphs),2):
#   rows_list.append(dbc.Row([
#       dbc.Col(dcc.Graph(figure=all_graphs[i])),
#       dbc.Col(dcc.Graph(figure=all_graphs[i+1])) if i+1 < len(all_graphs) else None
#       ])
#   )


app.layout = dbc.Container([
    html.H1(f'Layoff Data from {data_from_to_date}',className='text-center', style={'color':'red'}),
    dbc.Row([dbc.Col(card('Total Companies', total_companies_layoff)),dbc.Col(card('Total Layoffs', total_employees_layoff))]),

    dbc.Row([dbc.Col(dcc.Graph(figure=all_graphs[0])), dbc.Col(dcc.Graph(figure=all_graphs[1]))]),
    dbc.Row([dbc.Col(dcc.Graph(figure=all_graphs[2])), dbc.Col(dcc.Graph(figure=all_graphs[3]))]),
    dbc.Row([dbc.Col(dcc.Graph(figure=all_graphs[4])), dbc.Col(dcc.Graph(figure=all_graphs[6]))]),
    ])

# app.layout = dbc.Container([
#     html.H1(f'Layoff Data from {data_from_to_date}',className='text-center', style={'color':'red'}),
#     dbc.Row([dbc.Col(card('Total Companies', total_companies_layoff)),dbc.Col(card('Total Layoffs', total_employees_layoff))]),

#     dbc.Row([dbc.Col(dcc.Graph(figure=all_graphs[0])), dbc.Col(dcc.Graph(figure=all_graphs[1])),dbc.Col(dcc.Graph(figure=all_graphs[2]))]),
#     dbc.Row([dbc.Col(dcc.Graph(figure=all_graphs[3])), dbc.Col(dcc.Graph(figure=all_graphs[4])), dbc.Col(dcc.Graph(figure=all_graphs[6]))]),
#     ])

if __name__ == '__main__':
    app.run_server(mode='external',host="0.0.0.0",port=1005,debug=False)

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:1005
 * Running on http://172.28.0.12:1005
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Mar/2023 17:19:02] "GET /_alive_d4ae8d68-9aa2-4f8e-833c-6df63916b2ec HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>

In [26]:
# app2 = JupyterDash(__name__,external_stylesheets=[
#     "https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap-grid.min.css"
# ])



# card_content = lambda header, value: [
#     dbc.CardHeader(header),
#     dbc.CardBody(
#         [
#             html.H5(value, className="card-title"),
#         ]
#     ),
# ]

# cards = html.Div(
#     [
#         dbc.Row(
#             [
#                 dbc.Col(dbc.Card(card_content('total company participated in layoff',total_companies_layoff), color="primary", inverse=True)),
#                 dbc.Col(
#                     dbc.Card(card_content('total employees laid off', total_employees_layoff), color="secondary", inverse=True)
#                 ),
#             ],
#             className="mb-4",
#         )   
#     ]
# )

In [29]:
# https://www.datacamp.com/tutorial/learn-build-dash-python